In [1]:
from data import *
from PIL import Image
import os, shutil

In [2]:
crop("data/DRIVE_preprocessed/test/unet_result/original", "data/DRIVE_preprocessed/test/unet_result/cropped")

Cropping is done.


In [3]:
threshold("data/DRIVE_preprocessed/test/unet_result/cropped")

In [17]:
PATH = "C:/Users/Kaare/Documents/Git projects/unet"

In [18]:
TRAIN_PATH = PATH + "/data/DRIVE/training"

TEST_PATH = PATH + "/data/DRIVE/test"

PROCESSED_TRAIN_PATH = PATH + "/data/DRIVE_preprocessed/train"

PROCESSED_TEST_PATH = PATH + "/data/DRIVE_preprocessed/test"

In [19]:
shutil.rmtree(PATH + "/data/DRIVE_preprocessed", ignore_errors=False, onerror=None)
os.mkdir(PATH + "/data/DRIVE_preprocessed")
os.mkdir(PROCESSED_TRAIN_PATH)
os.mkdir(PROCESSED_TEST_PATH)
os.mkdir(PROCESSED_TRAIN_PATH + "/images")
os.mkdir(PROCESSED_TRAIN_PATH + "/labels")
os.mkdir(PROCESSED_TEST_PATH + "/images")

In [20]:
Nimgs = len(os.listdir(TRAIN_PATH + '/images'))
channels = 1
height = 584
width = 565

i=1
for file in sorted(os.listdir(TRAIN_PATH + "/images")):
  img = cv2.imread(TRAIN_PATH + "/images/"+file, 0)
  img = np.expand_dims(img, axis=0)
  img = np.moveaxis(img,0, -1)
  #print(img.shape)
  cv2.imwrite(PROCESSED_TRAIN_PATH + "/images/" + str(i) + '.png',img)
  i += 1


i=1
for file in sorted(os.listdir(TRAIN_PATH + "/labels")):
  g_truth = Image.open(TRAIN_PATH + "/labels/" + file)
  cv2.imwrite(PROCESSED_TRAIN_PATH + "/labels/" + str(i) + '.png', np.asarray(g_truth))
  i += 1


In [21]:
Nimgs = len(os.listdir(TEST_PATH + '/images'))
channels = 1
height = 584
width = 565

i=1
for file in sorted(os.listdir(TEST_PATH + "/images")):
  img = cv2.imread(TEST_PATH + "/images/"+file, 0)
  img = np.expand_dims(img, axis=0)
  img = np.moveaxis(img,0, -1)
  #print(img.shape)
  cv2.imwrite(PROCESSED_TEST_PATH + "/images/" + str(i) + '.png',img)
  i += 1

In [22]:
pad(PROCESSED_TRAIN_PATH + "/images", PROCESSED_TRAIN_PATH + "/images", False)
pad(PROCESSED_TRAIN_PATH + "/labels", PROCESSED_TRAIN_PATH + "/labels", False)
pad(PROCESSED_TEST_PATH + "/images", PROCESSED_TEST_PATH + "/images", False)

Padding is done.
Padding is done.
Padding is done.
